In [ ]:
# Import libraries
!pip install -U tensorflow tensorflow-datasets

import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
from tensorflow.keras import layers

train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

train_df["label_num"] = train_df.label.map({"ham":0, "spam":1})
test_df["label_num"] = test_df.label.map({"ham":0, "spam":1})

max_features = 10000
sequence_length = 100

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length
)

vectorize_layer.adapt(train_df["message"].values)
train_texts = tf.constant(train_df["message"].values)
train_labels = tf.constant(train_df["label_num"].values)
test_texts = tf.constant(test_df["message"].values)
test_labels = tf.constant(test_df["label_num"].values)

train_ds = tf.data.Dataset.from_tensor_slices((train_texts, train_labels)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_texts, test_labels)).batch(32)

train_ds = train_ds.map(lambda x, y: (vectorize_layer(x), y))
test_ds = test_ds.map(lambda x, y: (vectorize_layer(x), y))
model = keras.Sequential([
    layers.Embedding(max_features+1, 16),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
history = model.fit(train_ds, validation_data=test_ds, epochs=30)
def predict_message(pred_text):
    vectorized_text = vectorize_layer([pred_text])
    prediction = model.predict(vectorized_text)[0][0]
    label = "spam" if prediction > 0.5 else "ham"
    return [float(prediction), label]

# Probar con un ejemplo
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
